In [10]:
# Install microdf
!pip install git+https://github.com/PSLmodels/microdf.git
# update plotly
!pip install plotly --upgrade

  Cloning https://github.com/PSLmodels/microdf.git to /tmp/pip-req-build-l78w8axy
  Running command git clone -q https://github.com/PSLmodels/microdf.git /tmp/pip-req-build-l78w8axy
  Created wheel for microdf: filename=microdf-0.2.0-cp36-none-any.whl size=20086 sha256=4e11f4b2d4e40214f11c3562f391b6f794ee8ea6d2581e21b86ed80871be8bcd
  Stored in directory: /tmp/pip-ephem-wheel-cache-jkrsspzm/wheels/3d/53/af/92e56f83db191b0579d21e8385d61a92a502e66443b23c7e16
Successfully built microdf
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.14.1)


# Graph 1 - Child Poverty by Proportion Children Recieve of Adult UBI

In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import microdf as mdf

# import preprocessed data
summary2 = pd.read_csv("data/ratio_child_ben_to_adult_ben_summary.csv.gz",
                       compression='gzip')

# used to import data while drafting colab
# summary2 = pd.read_csv("https://github.com/fedderw/ubi-spending-on-children-blog-post/blob/main/jb/ratio_child_ben_to_adult_ben_summary.csv.gz?raw=true",
#                        compression='gzip')

# define UBI Center Colors
BLUE = '#1976D2'
DARK_BLUE = '#1565C0'
LIGHT_BLUE = '#90CAF9'
GRAY = '#BDBDBD'
BARELY_BLUE = '#E3F2FD'

# generate plotly graph
fig = px.line()

names = {0:"No UBI for children",
       1: "Half-sized UBI for children",
       2: "Full-sized UBI for children"}

colors = {0:DARK_BLUE,
       1: LIGHT_BLUE,
       2: GRAY}
       
percent_steps=[0,50,100]

zero=summary2[summary2['percent']==0]
fifty=summary2[summary2['percent']==50]
hundred=summary2[summary2['percent']==100]

dfs= [zero,fifty,hundred]

for i, df in enumerate(dfs):
  # add trace for optimal poverty df
  fig.add_trace(
      go.Scatter(
          x=df['funding_billions'],
          y=df['child_poverty_rate'],
          mode="markers+lines",
          name=names[i],
          # assign numpy array with child & adult ubi to customdata argument
          customdata = np.stack(
          (df['monthly_child_ubi'],
           df['monthly_adult_ubi']),
          axis=-1),
          # add customdata to hovertemplate
          hovertemplate = ('<i>Child Poverty Rate</i>: %{y:.1f}%<br>'+\
                        '<br><b>Child UBI/mo</b>: $%{customdata[0]: .0f}'+\
                        '<br><b>Adult UBI/mono </b>: $%{customdata[1]: .0f}<br>'),
                        # bonus hovertext if we want
                        # '<br><b>Poverty Rate </b>: %%{customdata[2]: .0f}<br>'+\
                        # '<br><b>Gini </b>: %{customdata[3]: .3f}<br>'+\
                        # '<br><b>Poverty Gap </b>: $%{customdata[4]: .0f}<br>'
                          
          line=dict(
              color=colors[i]
          )
      )
  )


fig.update_layout(
        title='Child Poverty by Proportion of Adult Benefit Paid to Children',
        xaxis_title='Funding in billions',
        yaxis_title='SPM child poverty rate',
        yaxis_ticksuffix='%',
        font=dict(family='Roboto'),
        hovermode='x',
        xaxis_tickprefix='$',
        xaxis_ticksuffix='B',
        plot_bgcolor='white',
        height=600,
        width=1000,
        legend_title_text='Benefit Design',
        margin_b=90 # add bottom margin for caption
        
    )

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.75
))

fig.update_traces(
  mode='markers+lines'
  )
config = {'displayModeBar': False}

fig.show(config=config)

# Graph 2 - Children's Share of Total UBI Spending

In [12]:
# import preprocessed data on simulation results

summary = pd.read_csv(
    "data/children_share_ubi_spending_summary.csv.gz", 
    compression='gzip')

# used to import data while drafting colab
# summary = pd.read_csv(
#     "https://github.com/fedderw/ubi-spending-on-children-blog-post/blob/main/jb/children_share_ubi_spending_summary.csv.gz?raw=true", 
#     compression='gzip')

# create dataframes for combinations of spending amounts that minimize
# poverty, minimize inequality, and maximize total winners 
optimal_poverty = summary.sort_values('poverty_gap').drop_duplicates('funding_billions', keep='first')
# drop rows where funding level is 0
optimal_poverty = optimal_poverty.drop(
    optimal_poverty[optimal_poverty.funding_billions==0].index
    ) 

optimal_inequality = summary.sort_values('gini').drop_duplicates('funding_billions', keep='first')
# drop rows where funding level is 0
optimal_inequality = optimal_inequality.drop(
    optimal_inequality[optimal_inequality.funding_billions==0].index
    ) 

optimal_winners = summary.sort_values('percent_better_off').drop_duplicates('funding_billions', keep='last')
optimal_winners = optimal_winners.drop(
    optimal_winners[optimal_winners.funding_billions==0].index
    ) 

# define adult population size, as determined in data preprocessing
adult_pop = 252117111.14000002
# define child population size
child_pop = 73151070.56999998
# calculate total population
pop = child_pop+adult_pop

# create plot
fig = px.line()

names = {0:"minimize poverty",
       1: "minimize inequality",
       2: "maximize winners"}

colors = {0:"blue",
       1: "black",
       2: "orange"}

# add poverty trace
fig.add_trace(
      go.Scatter(
          x=optimal_poverty['funding_billions'],
          y=optimal_poverty['percent'],
          mode="markers+lines",
          name="minimize poverty",
          # assign numpy array with child & adult ubi to customdata argument
          customdata = np.stack(
          (optimal_poverty['monthly_child_ubi'],
           optimal_poverty['monthly_adult_ubi'],
           optimal_poverty['poverty_rate'],
           optimal_poverty['gini'],
           optimal_poverty['poverty_gap']/1e9),
          axis=-1),
          # add customdata to hovertemplate
          hovertemplate = ('<i>Children\'s Share of Spending</i>: %{y:.0f}%<br>'+\
                        '<br><b>Child UBI/mo</b>: $%{customdata[0]: .0f}'+\
                        '<br><b>Adult UBI/mo </b>: $%{customdata[1]: .0f}<br>'
                         # bonus hovertext if we want
                        '<br><b>Poverty Rate </b>: %%{customdata[2]: .0f}<br>'
                        # '<b>Gini </b>: %{customdata[3]: .3f}<br>'+\
                        # '<b>Poverty Gap </b>: $%{customdata[4]: .0f} B<br>'
                          ),
          line=dict(
              color=DARK_BLUE
          )
      )
  )

# add inequality trace
fig.add_trace(
      go.Scatter(
          x=optimal_inequality['funding_billions'],
          y=optimal_inequality['percent'],
          mode="markers+lines",
          name="minimize Gini coeffecient",
          # assign numpy array with child & adult ubi to customdata argument
          customdata = np.stack(
          (optimal_inequality['monthly_child_ubi'],
           optimal_inequality['monthly_adult_ubi'],
           optimal_inequality['poverty_rate'],
           optimal_inequality['gini'],
           optimal_inequality['poverty_gap']/1e9),
          axis=-1),
          # add customdata to hovertemplate
          hovertemplate = ('<i>Children\'s Share of Spending</i>: %{y:.0f}%<br>'+\
                        '<br><b>Child UBI/mo</b>: $%{customdata[0]: .0f}'+\
                        '<br><b>Adult UBI/mo </b>: $%{customdata[1]: .0f}<br>'
                         # bonus hovertext if we want
                        # '<br><b>Poverty Rate </b>: %%{customdata[2]: .0f}<br>'+\
                        '<br><b>Gini </b>: %{customdata[3]: .3f}<br>'
                        # '<b>Poverty Gap </b>: $%{customdata[4]: .0f} B<br>'
                          ),
          line=dict(
              color=LIGHT_BLUE
          )
      )
  )

# add winners trace
fig.add_trace(
      go.Scatter(
          x=optimal_winners['funding_billions'],
          y=optimal_winners['percent'],
          mode="markers+lines",
          name="maximize winners",
          # assign numpy array with child & adult ubi to customdata argument
          customdata = np.stack(
          (optimal_winners['monthly_child_ubi'],
           optimal_winners['monthly_adult_ubi'],
           optimal_winners['poverty_rate'],
           optimal_winners['gini'],
           optimal_winners['poverty_gap']/1e9),
          axis=-1),
          # add customdata to hovertemplate
          hovertemplate = ('<i>Children\'s Share of Spending</i>: %{y:.0f}%<br>'+\
                        '<br><b>Child UBI/mo</b>: $%{customdata[0]: .0f}'+\
                        '<br><b>Adult UBI/mo </b>: $%{customdata[1]: .0f}<br>'
                         # bonus hovertext if we want
                        # '<br><b>Poverty Rate </b>: %%{customdata[2]: .0f}<br>'+\
                        # '<b>Gini </b>: %{customdata[3]: .3f}<br>'+\
                        # '<b>Poverty Gap </b>: $%{customdata[4]: .0f} B<br>'
                          ),
          line=dict(
              color=GRAY
          )
      )
  )



ratio = (child_pop / pop) * 100

fig.add_shape(
    type="line", line=dict(dash="dot", color="red"),
    x0=-1, x1=3000, y0=ratio, y1=ratio
)

# plot line where adult benefit is same size as child benefit
fig.add_annotation(
    text="Adult UBI = Child UBI",
    xref="paper",
    yref="paper",
    x=1,
    y=0.22,
    showarrow=False,
    font=dict(
              color="red",
              size=12
            ))

# CAPTION
# fig.add_annotation(
#     text="[CAPTION GOES HERE]",
#     xref="paper",
#     yref="paper",
#     x=0.0, 
#     y= -0.25,
    # showarrow=False,)

fig.update_xaxes(
        tickangle = 0,
        title_text = "Funding in billions",
        tickfont = {"size": 14},
        title_standoff = 25,
        ticksuffix='B',
        tickprefix='$',
        range=[0,3050])

fig.update_yaxes(
        title_text = "Children\'s Share of UBI Spending",
        ticksuffix ="%",
        tickfont = {'size':14},
        title_standoff = 25,
        range=[0,105])

fig.update_xaxes(title_font=dict(size=14, family='Roboto', color='black'))
fig.update_yaxes(title_font=dict(size=14, family='Roboto', color='black'))
fig.update_layout(
    title_text='Optimal share of UBI spending on children in order to:',
    hoverlabel_align = 'right', # new
    margin_b=90, # increase bottom margin to have space for caption
    # add_annotation
    # title = "Set hover text with hovertemplate"
    ) # new

fig.update_layout(plot_bgcolor='white', height=600, width=1000, hovermode="x")
fig.update_traces(
    mode='markers+lines',
    #  hovertemplate=None
     )

fig.show()
